# Qwen3-VL-30B-A3B-Instruct 推理流程详解

本 Notebook 详细讲解 Qwen3-VL-30B-A3B-Instruct 模型的完整推理流程，包括：
- **Vision Encoding**: 图像/视频的视觉编码
- **Prefill**: 首次处理完整输入序列
- **Decode**: 逐 token 自回归生成
- **MoE (Mixture of Experts)**: 稀疏专家路由机制

## 1. 模型架构总览

```
┌─────────────────────────────────────────────────────────────────┐
│                    Qwen3-VL-30B-A3B-Instruct                    │
├─────────────────────────────────────────────────────────────────┤
│  ┌─────────────────┐     ┌──────────────────────────────────┐  │
│  │  Vision Encoder │     │      Language Model (MoE)        │  │
│  │  ───────────────│     │  ────────────────────────────────│  │
│  │  • Patch Embed  │────▶│  • Token Embedding               │  │
│  │  • ViT Blocks   │     │  • Decoder Layers ×N             │  │
│  │  • Patch Merger │     │    - Self-Attention (mRoPE)      │  │
│  └─────────────────┘     │    - MoE FFN (128 experts, top8) │  │
│                          │  • LM Head                       │  │
│                          └──────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────┘
```

**关键参数:**
- 总参数: ~30B
- 激活参数: ~3B (每 token 只激活 8 个专家)
- 专家数量: 128 个 routed experts
- Top-K: 8

## 2. 完整推理流程

In [ ]:
# ============================================================
# 伪代码: Qwen3-VL-MoE 完整推理流程
# ============================================================

class Qwen3VLMoeForConditionalGeneration:
    """Qwen3-VL-MoE 模型主类"""
    
    def __init__(self):
        # 视觉编码器
        self.visual = Qwen3_VisionTransformer()
        # 语言模型 (MoE 架构)
        self.language_model = Qwen3MoeLLMForCausalLM()
    
    def forward(self, input_ids, positions, pixel_values=None, image_grid_thw=None):
        """
        主前向传播函数
        
        Args:
            input_ids: [batch_size, seq_len] 文本 token IDs
            positions: [3, seq_len] 或 [seq_len] 位置编码 (mRoPE)
            pixel_values: [N, C, H, W] 图像像素值
            image_grid_thw: [N, 3] 图像的 (T, H, W) 网格
        """
        # ========== 阶段1: 视觉编码 ==========
        if pixel_values is not None:
            # 对图像进行编码
            vision_embeddings = self.visual(pixel_values, image_grid_thw)
            # shape: [num_patches, hidden_size]
            
            # 将视觉 embedding 合并到文本 embedding 中
            inputs_embeds = self._merge_multimodal_embeddings(
                input_ids, vision_embeddings
            )
        else:
            inputs_embeds = self.language_model.embed_tokens(input_ids)
        
        # ========== 阶段2: LLM 前向传播 (包含 MoE) ==========
        hidden_states = self.language_model.model(
            input_ids=None,
            positions=positions,
            inputs_embeds=inputs_embeds,
        )
        
        # ========== 阶段3: 输出预测 ==========
        logits = self.language_model.lm_head(hidden_states)
        return logits

## 3. Vision Encoder 详解

In [ ]:
class Qwen3_VisionTransformer:
    """视觉编码器 - 将图像转换为 embedding"""
    
    def __init__(self):
        self.patch_embed = PatchEmbed3D()      # 3D 卷积 patch embedding
        self.blocks = [VisionBlock() for _ in range(depth)]  # ViT blocks
        self.merger = PatchMerger()            # 空间下采样
    
    def forward(self, x, grid_thw):
        """
        Args:
            x: [N, C, T, H, W] 输入图像/视频
               - N: batch 中的图像数量
               - C: 通道数 (3 for RGB)
               - T: 时间维度 (图像=1, 视频>1)
               - H, W: 高度和宽度
            grid_thw: [N, 3] 每个图像的 (T, H, W) 网格尺寸
        
        Returns:
            hidden_states: [total_patches, hidden_size]
        """
        # Step 1: Patch Embedding (3D卷积)
        # 将图像分割成 14x14 的 patches
        hidden_states = self.patch_embed(x)
        # shape: [total_patches, hidden_size]
        # 例如 448x448 图像 -> (448/14)^2 = 1024 patches
        
        # Step 2: 添加位置编码 (2D RoPE)
        pos_embeds = self.compute_position_embedding(grid_thw)
        hidden_states = hidden_states + pos_embeds
        
        # Step 3: Vision Transformer Blocks
        for block in self.blocks:
            hidden_states = block(
                hidden_states,
                rotary_pos_emb=self.rope_embed,
            )
        
        # Step 4: Patch Merger (空间下采样 2x2)
        # 将相邻的 2x2 patches 合并为 1 个
        hidden_states = self.merger(hidden_states)
        # patches 数量减少 4 倍: 1024 -> 256
        
        return hidden_states

# Vision 输出示例:
# 输入: 448x448 RGB 图像
# Patch Embed 后: [1024, 1280] (1024 patches, 1280-dim)
# Merger 后: [256, 3584] (256 tokens, 3584-dim 匹配 LLM hidden_size)

## 4. MoE Layer 详解

In [ ]:
class Qwen3MoeSparseMoeBlock:
    """
    MoE (Mixture of Experts) 层
    
    核心思想: 不是所有 token 都经过所有参数
    每个 token 只选择 top-K 个专家进行计算
    """
    
    def __init__(self):
        self.num_experts = 128          # 总专家数
        self.top_k = 8                  # 每 token 激活的专家数
        self.gate = Linear(hidden_size, num_experts)  # 路由器
        self.experts = FusedMoE(...)    # 128 个 FFN 专家
    
    def forward(self, hidden_states):
        """
        Args:
            hidden_states: [num_tokens, hidden_size]
        
        Returns:
            output: [num_tokens, hidden_size]
        """
        num_tokens, hidden_dim = hidden_states.shape
        
        # ========== Step 1: Router/Gate 计算 ==========
        # 计算每个 token 对每个专家的偏好分数
        router_logits = self.gate(hidden_states)
        # shape: [num_tokens, num_experts=128]
        
        # ========== Step 2: Top-K 选择 ==========
        # 对每个 token, 选择得分最高的 K 个专家
        topk_weights, topk_ids = fused_topk(
            hidden_states,
            router_logits,
            topk=8,
            renormalize=True,  # 权重归一化
        )
        # topk_weights: [num_tokens, 8] 专家权重
        # topk_ids: [num_tokens, 8] 专家 ID
        
        # ========== Step 3: Expert FFN 计算 ==========
        # 使用 Fused MoE Kernel 高效计算
        output = self.experts(
            hidden_states=hidden_states,
            router_logits=router_logits,
        )
        # 内部流程:
        # 1. 按专家分组 tokens
        # 2. 并行计算各专家的 FFN
        # 3. 加权聚合结果
        
        return output  # [num_tokens, hidden_size]

### 4.1 MoE 路由可视化

```
Token 1: "Hello"  ─── Router ───▶ Expert 3 (0.35), Expert 17 (0.25), ... (top-8)
Token 2: "World"  ─── Router ───▶ Expert 5 (0.40), Expert 3 (0.20), ... (top-8)
Token 3: [IMAGE]  ─── Router ───▶ Expert 42 (0.30), Expert 8 (0.28), ... (top-8)
    │
    ▼
┌────────────────────────────────────────────────────────────────┐
│                     Expert Pool (128 experts)                  │
├────────────────────────────────────────────────────────────────┤
│ Expert 0  │ Expert 1  │ ... │ Expert 42 │ ... │ Expert 127    │
│ (idle)    │ (idle)    │     │ (active)  │     │ (idle)        │
└────────────────────────────────────────────────────────────────┘
    │
    ▼
加权聚合: output = Σ(weight_i × expert_i(hidden_states))
```

## 5. Prefill vs Decode 阶段对比

In [ ]:
# ============================================================
# Prefill 阶段: 一次性处理完整输入
# ============================================================

def prefill_phase(model, prompt_tokens, image):
    """
    Prefill: 处理用户输入的完整 prompt + 图像
    
    特点:
    - 输入序列较长 (prompt + image patches)
    - 计算密集型 (compute-bound)
    - Self-attention 是完整的 causal attention
    - 生成 KV Cache 供 decode 阶段使用
    """
    # 输入示例
    prompt = "Describe this image in detail."
    seq_len = len(prompt_tokens) + num_image_patches  # 例如: 50 + 256 = 306
    
    # Vision Encoding (只在 prefill 执行一次)
    vision_embeds = model.visual(image, grid_thw)
    # shape: [256, hidden_size]
    
    # 合并 text + vision embeddings
    inputs_embeds = merge_embeddings(prompt_tokens, vision_embeds)
    # shape: [306, hidden_size]
    
    # 通过所有 decoder layers (包含 MoE)
    # Attention: [306, 306] 的 causal mask
    hidden_states = model.forward(inputs_embeds, positions)
    
    # 生成第一个 token
    first_token_logits = model.lm_head(hidden_states[-1])
    first_token = sample(first_token_logits)
    
    # 保存 KV Cache
    kv_cache = save_kv_cache()  # [num_layers, 2, seq_len, num_heads, head_dim]
    
    return first_token, kv_cache

In [ ]:
# ============================================================
# Decode 阶段: 逐 token 生成
# ============================================================

def decode_phase(model, kv_cache, last_token):
    """
    Decode: 自回归生成, 每次只处理 1 个新 token
    
    特点:
    - 输入序列长度 = 1 (单个新 token)
    - 内存密集型 (memory-bound)
    - 利用 KV Cache 避免重复计算
    - Attention 只计算新 token 与所有历史 token
    """
    # 输入: 上一步生成的 token
    new_token_embed = model.embed_tokens(last_token)
    # shape: [1, hidden_size]
    
    # Self-Attention 使用 KV Cache
    # Query: [1, num_heads, head_dim] (只有新 token)
    # Key:   [seq_len+1, num_heads, head_dim] (历史 + 新)
    # Value: [seq_len+1, num_heads, head_dim]
    
    # MoE 计算 (仍然每个 token 选 top-8 专家)
    # 但因为只有 1 个 token, 计算量大大减少
    hidden_states = model.forward(
        new_token_embed, 
        position=current_pos,
        kv_cache=kv_cache,
    )
    
    # 预测下一个 token
    next_token_logits = model.lm_head(hidden_states)
    next_token = sample(next_token_logits)
    
    # 更新 KV Cache
    kv_cache = update_kv_cache(kv_cache, new_kv)
    
    return next_token, kv_cache

# Decode 循环
def generate(model, prompt, image, max_tokens=100):
    # Prefill
    token, kv_cache = prefill_phase(model, prompt, image)
    generated = [token]
    
    # Decode loop
    for _ in range(max_tokens):
        token, kv_cache = decode_phase(model, kv_cache, token)
        generated.append(token)
        if token == EOS_TOKEN:
            break
    
    return generated

### 5.1 Prefill vs Decode 对比表

| 特性 | Prefill | Decode |
|------|---------|--------|
| 输入长度 | 整个 prompt + 图像 patches | 1 个 token |
| 计算特点 | Compute-bound | Memory-bound |
| Attention | Full causal (NxN) | Query=1, KV=历史 |
| KV Cache | 生成并保存 | 读取并更新 |
| Vision Encoder | 执行 | 跳过 |
| 执行次数 | 1 次 | 每个生成 token 1 次 |

## 6. Decoder Layer 完整流程

In [ ]:
class Qwen3MoeDecoderLayer:
    """单个 Decoder Layer 的完整流程"""
    
    def __init__(self):
        self.input_layernorm = RMSNorm(hidden_size)
        self.self_attn = Qwen3MoeAttention()  # 带 QK Norm 的注意力
        self.post_attention_layernorm = RMSNorm(hidden_size)
        self.mlp = Qwen3MoeSparseMoeBlock()  # MoE 层
    
    def forward(self, positions, hidden_states, residual):
        """
        Pre-LN Transformer 架构 (LN 在子层之前)
        """
        # ========== Self-Attention 部分 ==========
        if residual is None:
            residual = hidden_states
            hidden_states = self.input_layernorm(hidden_states)
        else:
            hidden_states, residual = self.input_layernorm(
                hidden_states, residual
            )
        
        # Self-Attention with mRoPE (多分辨率旋转位置编码)
        hidden_states = self.self_attn(
            positions=positions,     # [3, seq_len] for mRoPE
            hidden_states=hidden_states,
        )
        
        # ========== MoE FFN 部分 ==========
        hidden_states, residual = self.post_attention_layernorm(
            hidden_states, residual
        )
        
        # MoE: 稀疏专家计算
        hidden_states = self.mlp(hidden_states)
        # 每个 token 只激活 8/128 = 6.25% 的专家参数
        
        return hidden_states, residual

## 7. FusedMoE 内部实现

In [ ]:
def fused_moe_forward(hidden_states, router_logits, w1, w2, topk=8):
    """
    Fused MoE 的核心计算流程 (简化版)
    
    实际实现使用 Triton kernel 进行融合优化
    """
    num_tokens, hidden_dim = hidden_states.shape
    num_experts = router_logits.shape[-1]  # 128
    
    # Step 1: TopK Routing
    # 选择每个 token 的 top-8 专家
    topk_weights = softmax(router_logits, dim=-1)  # 归一化
    topk_weights, topk_ids = topk(topk_weights, k=8)
    topk_weights = normalize(topk_weights)  # 重新归一化
    
    # Step 2: Token-to-Expert 分配
    # 按专家分组 tokens (便于并行计算)
    # sorted_token_ids: 按专家排序的 token 索引
    # expert_ids: 对应的专家 ID
    sorted_token_ids, expert_ids = moe_align_block_size(
        topk_ids, block_size=128
    )
    
    # Step 3: Expert FFN 计算 (Gate-Up-Down)
    # 对每个专家:
    #   gate = w1_gate @ hidden_states  # 门控
    #   up = w1_up @ hidden_states       # 上投影
    #   hidden = SiLU(gate) * up         # 激活
    #   output = w2 @ hidden             # 下投影
    
    # Triton 融合内核: 一次 kernel launch 完成上述计算
    expert_outputs = fused_moe_kernel(
        A=hidden_states,           # 输入
        B=w1,                       # 专家权重 [E, N, K]
        sorted_token_ids=sorted_token_ids,
        expert_ids=expert_ids,
        topk_weights=topk_weights,
    )
    
    # Step 4: 加权聚合
    # output[i] = Σ_k (weight[i,k] × expert_output[i,k])
    output = weighted_sum(expert_outputs, topk_weights)
    
    return output

# 实际内存节省:
# Dense FFN: 所有 token × 所有参数 = O(N × hidden × intermediate)
# Sparse MoE: 所有 token × top-K 参数 = O(N × hidden × intermediate × K/E)
# 节省比例: 8/128 = 6.25%

## 8. 总结: 完整数据流

```
输入: "描述这张图片" + [IMAGE]
        │
        ▼
┌─────────────────────────────────────────────────────────────────┐
│  Tokenizer: 文本 → Token IDs [1, 2, 3, 4, 5, <IMG>, 7, 8, ...]  │
└─────────────────────────────────────────────────────────────────┘
        │
        ├────────────────────────────────────────┐
        ▼                                        ▼
┌─────────────────────┐              ┌─────────────────────┐
│  Token Embedding    │              │   Vision Encoder    │
│  [5, hidden_size]   │              │   [256, hidden_size]│
└─────────────────────┘              └─────────────────────┘
        │                                        │
        └────────────────┬───────────────────────┘
                         ▼
           ┌───────────────────────────┐
           │  Merge: [261, hidden_size] │ ◄── Prefill 输入
           └───────────────────────────┘
                         │
         ┌───────────────┴───────────────┐
         ▼                               ▼
┌─────────────────┐           ┌─────────────────────────────┐
│ Self-Attention  │           │         MoE Layer           │
│ (with mRoPE)    │───────────▶  Router → Top-8 → Experts   │
│ [261, hidden]   │           │  [261, hidden] → [261, hidden]│
└─────────────────┘           └─────────────────────────────┘
                                         │
                              × N layers (循环)
                                         │
                                         ▼
                         ┌───────────────────────┐
                         │  LM Head → Logits     │
                         │  [261, vocab_size]    │
                         └───────────────────────┘
                                         │
                                         ▼
                              取最后一个 token logits
                                         │
                                         ▼
                         ┌───────────────────────┐
                         │  Sampling → "这"       │ ◄── 第一个生成 token
                         └───────────────────────┘
                                         │
                              Decode 循环...
                                         │
                                         ▼
输出: "这张图片展示了一只可爱的小猫..."
```

## 9. 相关源码位置

| 组件 | 文件路径 | 关键类/函数 |
|------|---------|-------------|
| 主模型 | `vllm/model_executor/models/qwen3_vl_moe.py` | `Qwen3VLMoeForConditionalGeneration` |
| Vision Encoder | `vllm/model_executor/models/qwen3_vl.py` | `Qwen3_VisionTransformer` |
| MoE Block | `vllm/model_executor/models/qwen3_moe.py` | `Qwen3MoeSparseMoeBlock` |
| FusedMoE Layer | `vllm/model_executor/layers/fused_moe/layer.py` | `FusedMoE` |
| MoE Kernels | `vllm/model_executor/layers/fused_moe/fused_moe.py` | `fused_topk`, `invoke_fused_moe_kernel` |